In [ ]:
# # Calculate FC on source signals 
# Things to check before running: 
# path to preprocessed EED data, original srate, downsampled srate, FOI, 
# epoch length and overlap (for both wPLI and AEC), FC metric, N of cycles per wavelet, name and path of the results file

In [ ]:
import mne
import numpy as np
from scipy.io import loadmat
from mne_connectivity import spectral_connectivity_time, envelope_correlation
import os
import pandas as pd
rng = np.random.default_rng(42)  # set seed for reproducibility

In [ ]:
# Define the folder path for preprocessed EEG data
data_folder = r"W:\Projects\2019-04 M1M1PAS Project\analysis\source_signals\Beamformer"

# Get a list of all the EEG files
file_list = [f for f in os.listdir(data_folder)]

# Initialize an empty list to store results
data = []

# Loop through each file in the folder
for file in file_list:
    
    # Get meta data from the file name
    subject = file.split('_')[0].replace('sub-', '')
    task = file.split('_')[1].replace('task-', '')
    run = file.split('_')[2].replace('run-', '')
    
    # Load current file
    file_path = os.path.join(data_folder, file)
    matlabdata = loadmat(file_path)
    
    # Extract source time courses
    leftSM = matlabdata['leftSM']
    rightSM = matlabdata['rightSM']

    # Create a new MNE RawArray with the raw data
    info = mne.create_info(ch_names=['leftSM', 'rightSM'], sfreq=1000, ch_types='eeg')
    source_data = np.vstack([leftSM, rightSM])
    source_raw = mne.io.RawArray(source_data, info)
    
    # Downsample from 1k to 200Hz
    source_raw.resample(sfreq=200)
    
    # Define the FOI 8-13 HZ
    fmin, fmax = 8., 13.
    freqs = np.linspace(fmin, fmax, int((fmax - fmin) * 2 + 1))
    
    # wPLI
    # Epoch data
    epochs_raw = mne.make_fixed_length_epochs(source_raw, duration=2, overlap=1.5, preload=True)
    
    # Calculate FC
    con = spectral_connectivity_time(
        epochs_raw,
        method=['wpli'],  # Select wPLI as a metric
        freqs=freqs,
        faverage=True,  # Average connectivity across frequencies within the band
        mode='cwt_morlet',  # Use Morlet wavelet
        n_cycles=5, # 5 cycles in the wavelet
        average=False, # Retain epochs
        n_jobs=1 
    )
    wpli = con.get_data()[:,2,0]
    
    # Collect the data
    data.extend([(subject, task, run, wpli_val) for wpli_val in wpli])

# Convert the list to a dataframe
datapd = pd.DataFrame(data, columns=['subject','task', 'run', 'wpli'])

# Save results
new_filename = f"sourceConnectivity.csv"
new_file_path = os.path.join(r"W:\Projects\2019-04 M1M1PAS Project\analysis\source_signals\fc_source",new_filename)
datapd.to_csv(new_file_path)
